In [1]:
# Import Libraries
import sys
import math
import glob, os
import cv2
import numpy as np
import pandas as pd
import geopy.distance
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import skimage.measure
from skimage import exposure
from skimage import feature


2024-07-16 11:38:31.531961: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-16 11:38:32.596462: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdjkhosh/.local/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2024-07-16 11:38:32.596584: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdjkhosh/.local/lib/python3.10/sit

In [2]:
# Setup paths and import local libraries
%cd ~/Projects/
%cd SkyAI-Sim

prj_path = Path('SkyAI-Sim')
sys.path.insert(0, str(prj_path))
from src.utils import geo_helper, io_helper, img_helper
from src.utils.io_helper import pretty, import_module
from src.utils import consts
from src.data.googlemap import GoogleMap

args = consts.ARGS
args

/home/sdjkhosh/Projects
/home/sdjkhosh/Projects/SkyAI-Sim
Namespace(coords='dataset/sample_coords.txt', fov=78.8, aspect_ratio=[4, 3], overlap=0, utm='EPSG:32616', data_dir='dataset/Test', vmargin=20, img_size=[400, 400, 3], batch_size=8, seed=2024)


usage: SkyAI Sim Project [-h] [--coords COORDS] [--fov FOV]
                         [--aspect_ratio ASPECT_RATIO [ASPECT_RATIO ...]]
                         [--overlap OVERLAP] [--utm UTM] [--data_dir DATA_DIR]
                         [--vmargin VMARGIN]
                         [--img_size IMG_SIZE [IMG_SIZE ...]]
                         [--batch_size BATCH_SIZE] [--seed SEED]
SkyAI Sim Project: error: argument --fov: invalid float value: '/home/sdjkhosh/.local/share/jupyter/runtime/kernel-v2-645nreOXFoQrj4Y.json'


Namespace(coords='dataset/sample_coords.txt', fov=78.8, aspect_ratio=[4, 3], overlap=0, utm='EPSG:32616', data_dir='dataset/Test', vmargin=20, img_size=[400, 400, 3], batch_size=8, seed=2024)

In [3]:
coords_list = np.asanyarray(pd.read_csv(args.coords, dtype=float, sep=' '))
coords_list

array([[ 3.40522350e+01, -1.18243683e+02,  1.00000000e+02],
       [ 4.07127760e+01, -7.40059740e+01,  5.00000000e+01],
       [ 5.15073510e+01, -1.27758000e-01,  2.00000000e+02],
       [ 3.56894870e+01,  1.39691711e+02,  1.50000000e+02],
       [ 4.88566130e+01,  2.35222200e+00,  7.50000000e+01],
       [ 5.57558250e+01,  3.76172980e+01,  1.20000000e+02],
       [-3.38688200e+01,  1.51209290e+02,  1.80000000e+02],
       [-2.35505200e+01, -4.66333080e+01,  9.00000000e+01],
       [ 3.77749290e+01, -1.22419418e+02,  1.30000000e+02]])

In [4]:
for coords in coords_list:
        print(coords)
        bbox_m = geo_helper.get_map_dim_m(
                args.fov, coords[-1],
                args.aspect_ratio[0]/args.aspect_ratio[1]
                )
        bbox = geo_helper.calc_bbox_m(coords[:2],
                                        bbox_m)
        args.coords = tuple(np.array(bbox).flatten()) + (coords[-1],)
        aerial_data = GoogleMap(
        args=args,
        map_type='satellite',
        data_dir=args.data_dir,
        overlap=args.overlap
        )
        aerial_data.check_data()

[  34.052235 -118.243683  100.      ]


2024-07-16 11:38:34.650479: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-16 11:38:34.650526: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: c2-parisa


Checking folder:
	 dataset/Test/satellite_0 Folder Exists.

----------------------------------------------------------
                      Info:Warning!                       
----------------------------------------------------------

Changing UTM Zone to EPSG:32611

----------------------------------------------------------



2024-07-16 11:38:34.650538: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: c2-parisa
2024-07-16 11:38:34.650667: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.54.0
2024-07-16 11:38:34.650697: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.54.0
2024-07-16 11:38:34.650705: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.54.0
2024-07-16 11:38:34.651572: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



---------------------------------------------------------------------------
        Info:<src.data.googlemap.GoogleMap object at 0x7f49ecc83550>       
---------------------------------------------------------------------------

[INFO] Data detailed values before download.
 TL = Top Left, BR = Bottom Right
                             y/lat       x/lon  z/agl total/area                                                                                                                                                                                                                                                     more info
TL                       34.05237   -118.2439                                                                                                                                                                                                                                                                                
BR                        34.0521 -118.243466         

In [5]:
128 + -0.128028 * (256/360)


127.90895786666667

In [6]:
from src.utils.consts import pixel_per_degree, tile_center_p
lon = -0.12802783012523197

tile_center_p['x'] + lon * pixel_per_degree

127.90895798746651